#Extracting Arabic keywords from social media posts

This notebook provides two examples of how to extract Arabic keywords from social media posts using the Python KeyBERT package. KeyBERT is a simple and easy-to-use keyword extraction technique that uses BERT embeddings to generate keywords and keyphrases that are most similar to a document. However, extracting keywords from social media posts is not as straightforward as extracting from documents, and existing tools do not work as well with Arabic as they do with English.



In [ ]:
# install  packages

!pip install keybert
!pip install keyphrase-vectorizers
!pip install pyarabic
!pip install Arabic-stopwords
!pip install camel-tools


Data processing

In [ ]:
import re
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
import arabicstopwords.arabicstopwords as stpw


def text_processing(text):
  # remove non arabic data
  #text=tokenize(text, conditions=is_arabicrange, morphs=strip_tashkeel)
  text=tokenize(text,  morphs=strip_tashkeel)

  #remove arabic stopwords
  stp_words_list = open("./arabicstopword.txt", encoding="utf-8")
  stp_words = stp_words_list.read()
  stp_words = stp_words.split("\n")
  from camel_tools.utils.normalize import(
    normalize_alef_ar,
    normalize_alef_maksura_ar,
    normalize_teh_marbuta_ar,
    )
  added = []

  for word in stp_words:
    nar = normalize_alef_ar(word)
    nam = normalize_alef_maksura_ar(nar)
    res = normalize_teh_marbuta_ar(word)
    added.append(res)

  stp_words.append(added)
  tokens = [w for w in text if not stpw.is_stop(w)]
  tokens = [word for word in text if not word in stp_words]
  return tokens


I'm going to implement the extraction function with KeyBERT using two different classes:

1.   CountVectorizer
2.   KeyphraseCountVectorizer


##1- using CounterVectorize

The default model in KeyBERT ("all-MiniLM-L6-v2") works great for English documents. In contrast, for multi-lingual documents or any other language, "paraphrase-multilingual-MiniLM-L12-v2"" has shown better performance.
However, using (paraphrase-mpnet-base-v2) and paraphrase-multilingual-mpnet-base-v2 perofrm well but with higher complixity.



In [ ]:
#==================================================
#1- using CounterVectorize
#==================================================

def extract_non_verb_keywords(sentence):
  from transformers import pipeline
  pos = pipeline('token-classification', model='CAMeL-Lab/bert-base-arabic-camelbert-mix-pos-msa')
  pos=pos(sentence)
  non_verb_kw=[]
  for i in range (len(pos)):
    kw=[]
    for element in pos[i]:
      if element.get('entity') not in ['verb', 'prep']:
       w = element.get('word')
       kw.append(w)
       listToStr = ' '.join([str(elem) for elem in kw])
       listToStr=listToStr.replace(" ##","")
    non_verb_kw.append(listToStr)
  return non_verb_kw

In [ ]:
def arabic_keyword_extract_CV(text):
  from keybert import KeyBERT
  from sklearn.feature_extraction.text import CountVectorizer
  #processing data
  tokens=text_processing(text5)
  text=" ".join(tokens)
  # with CountVectorizer, we should to define manually the keyword length using ngram_range
  vectorizer= CountVectorizer(ngram_range=(1, 3))

  kw_extractor = KeyBERT('paraphrase-multilingual-mpnet-base-v2')
  keywords = kw_extractor.extract_keywords(text , vectorizer=vectorizer, stop_words="arabic", use_mmr=True, diversity=0.7, top_n=5)
  keywords=[item[0] for item in keywords]

  #remove verbs
  non_verb_keywords= extract_non_verb_keywords(keywords)
  import re
  # Define a regular expression pattern for Arabic letters
  keywods= [keword for keword in non_verb_keywords if re.search('[\u0600-\u06FF\u0750-\u077F]+', keword)]
  output = dict()
  output["original_text"] = text
  output["keywords"] = [','.join(keywods)]
  import pandas as pd
  df = pd.DataFrame(output)
  df.to_csv("arabic_keywords_CV.csv")
  return output

#testing
text="لتكن عبرة للاخرين اصبحنا نرى انتشار واسع للتافهين بمواقع التواصل الاجتماعي واصبحو حديث الكل رغم انهم لم يقدمو اي قيمة مضافة للمجتمع لاكن للاسف فان النجوم الحقيقين الذين يستحقون ان يتصدرو الطوندونس نكاد لانسمع عنهم شيء رغم انهم يحتلونة مكانة رفيعة في اكبر المؤسسات العلمية الاروبية والعالمية .,ar"
text2=" أخي الفلاح، أختي الفلاحة، القرض الفلاحي للمغرب كينخارط بجنبكم في المبادرة الوطنية للتأمين الاجباري عن المرض لضمان صحتكم وصحة عائلتكم. وهذا هو الوقت المناسب باش يكون عندكم حساب شخصي مستقل على حسابكم المهني"
text3="من 15 غشت إلى 18 سبتمبر 2022، ادفعوا لأول مرة بدون تماس باستخدام بطاقة Visa الخاصة بكم في المتاجر Maârif Culture , GO Sport Maroc , City Sport Maroc , PUMA Maroc Courir Maroc : وDaha Fazla استفيدوا من تعويض ٪20 😍 !للعلم أن جميع عمليات الدفع بدون تماس في هذه العلامات التجارية بواسطة بطاقة Visa تسمح لكم بالمشاركة تلقائيًا في السحب الكبير 🎉 لمحاولة الفوز بهدايا رائعة 🎁 بما في ذلك سيارة !المزيد من المعلومات عبر الرابط التالي : https://bit.ly/3pf9ARC "
text4="بمناسبة الذكرى 43 لاسترجاع وادي الذهب، يتقدم القرض الفلاحي للمغرب بأحر التهاني لكافة الشعب المغربي"
text5="بمناسبة اليوم الوطني للجالية المغربية بالخارج، القرض الفلاحي للمغرب يرحب بكم. ولمرافقتكم اليومية ، القرض الفلاحي للمغرب صمّم من اجلكم Air Pack MDM #RESPIRE عرض بنكي شامل يلائم متطلباتكم و يفتح أمامكم أبواب عالم من الامتيازات الجديدة . "
outputs = arabic_keyword_extract_CV (text)
print (outputs)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-pos-msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'original_text': 'بمناسبة الوطني للجالية المغربية بالخارج القرض الفلاحي للمغرب يرحب بكم . ولمرافقتكم اليومية القرض الفلاحي للمغرب صمم اجلكم Air Pack MDM # RESPIRE عرض بنكي شامل يلائم متطلباتكم يفتح أمامكم أبواب عالم الامتيازات الجديدة .', 'keywords': ['القرض الفلاحي للمغرب,القرض الفلاحي للمغرب,الوطني']}


 ## 2- *KeyphraseCountVectorizer*

In [ ]:

#==================================================
# 2- using KeyphraseCountVectorizer
#==================================================

from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

def arabic_keyword_extract_KCV(text):

  import re
  text = re.sub('([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', text)
  #pos_pattern regexp to ensure extracting keywords which start with nouns
  vectorizer = KeyphraseCountVectorizer()
  #try:
  keywords = []
  kw_extractor = KeyBERT("paraphrase-multilingual-mpnet-base-v2")
  keywords = kw_extractor.extract_keywords(text , keyphrase_ngram_range=(2,3), vectorizer=vectorizer, stop_words="arabic", top_n=3, use_mmr=True, diversity=0.7)
  keywords = [ x[0] for x in keywords]
  output = dict()
  output["original_text"] = text
  output["keywords"] = [','.join(keywords)]
  import pandas as pd
  df = pd.DataFrame(output)
  df.to_csv("arabic_keywords_KCV.csv")
  return output

In [ ]:

#testing with 5 posts:

outputs= arabic_keyword_extract_KCV(text)
print(outputs)



{'original_text': 'لتكن عبرة للاخرين اصبحنا نرى انتشار واسع للتافهين بمواقع التواصل الاجتماعي واصبحو حديث الكل رغم انهم لم يقدمو اي قيمة مضافة للمجتمع لاكن للاسف فان النجوم الحقيقين الذين يستحقون ان يتصدرو الطوندونس نكاد لانسمع عنهم شيء رغم انهم يحتلونة مكانة رفيعة في اكبر المؤسسات العلمية الاروبية والعالمية ', 'keywords': ['التواصل الاجتماعي واصبحو حديث الكل رغم انهم,اصبحنا نرى انتشار واسع للتافهين,لتكن عبرة']}
